In [1]:
#Installing the library
#!pip install requests
#!pip install bs4

In [1]:
#Importing the libraries
from requests import get
from bs4 import BeautifulSoup

Requesting content from webpage- https://www.flipkart.com/apple-iphone-14-starlight-128-gb/product-reviews/itm3485a56f6e676?pid=MOBGHWFHABH3G73H&lid=LSTMOBGHWFHABH3G73HVXY5AV&marketplace=FLIPKART


Here in the next cell, 
1. Importing the get() function from the requests module. 
2. Assigning the address of the webpage to a variable named URL.
3. Request the server the content of the webpage by using get(), and store the server's response in the variable RESPONSE.
4. Print a small part of response's content by accessing its .text attribute (response is now a Response object).

In [2]:
#1 Importing get() function done above.
#2 Assigning the address of the webpage to a variable URL

URL = 'https://www.flipkart.com/apple-iphone-14-starlight-128-gb/product-reviews/itm3485a56f6e676?pid=MOBGHWFHABH3G73H&lid=LSTMOBGHWFHABH3G73HVXY5AV&marketplace=FLIPKART'

#3 Requesting the server the content of the webpage by using get() and storing the server's response in the variable RESPONSE.
response = get(URL, verify = False)

#4 Printing the small part of response by using .text attribute

print(response.text[:500])

C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<!doctype html><html lang="en"><head><link href="https://rukminim2.flixcart.com" rel="preconnect"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.905c37.css"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.ccbde3.css"/><meta http-equiv="Content-type" content="text/html; charset=utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=Edge"/><meta property="fb:page_id" content


Using BeautifulSoup to parse the HTML content: (Parsing is the method of translating code into machine language to investigate the appropriate syntax of the code.)
=> To parse our HTML document and extract the 50 div containers, we'll use a python module called BeautifulSoup, the most common web scraping module for python.

In the following code cell we will
1. Import the BeautifulSoup class creator from the package bs4.
2. Parse response.text by creating a BeautifulSoup object, and assign this object to html_soup. The "html.parser" indicates that we want to do the parsing using Python's built-in HTML parser. 

In [3]:
# Using BeautifulSoup to create a class creator

html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

Before extracting the 50 div containers we need to figure out what distinguishes them from other div elements on that page.

Often, the distinctive mark resides in the class attribute. If you inspect the HTML lines of the containers of interest, you'll notice that the class attribute has two values: "col _2wzgFH" and "K0kLPL".

The combination is unique to these div containers. We can see that's true by doing a quick search (Ctrl +F). We have 50 such containers so we expect to see only 10 matches:

In [9]:
# Using find_all() method to extract all the div containers that have a class attribute of "col _2wzgFH K0kLPL"

review_containers = html_soup.find_all('div', class_ ='col _2wzgFH K0kLPL')

## Printing type and length of this container

print(type(review_containers))
print(len(review_containers))

<class 'bs4.element.ResultSet'>
10


The find_all() command returned a ResultSet object which is a list containing all the 10 divs we are interested in.

Now we'll select only the first container, and extract, by turn, each item of interest:
1. The name of the reviewer
2. Subject
3. The review
4. The rating
5. The no of likes
6. The no of dislikes

In [10]:
# Lets try finding the name of the reviewer from this review_containers which contain the details about all 10 reviewers on first page

# Find the detail of first review
first_review = review_containers[0]
first_review.div

#3 Accessing the first review

first_review.find('div', class_ = '').text

#2 Subject: 
first_review.p.text

#1 Name of the reviewer

first_review.find('p', class_ = '_2sc7ZR _2V5EHH').text

'Rahul  Pandey'

In [11]:
#4 The rating
first_review.find('div', class_ = '_3LWZlK _1BLPMq').text

'5'

In [12]:
#5 The no of likes
first_review.find('span', class_ = '_3c3Px5').text

'85'

In [13]:
#6 The no of dislikes
first_review.find('div', class_ = '_1LmwT9 pkR4jH').find('span', class_ = '_3c3Px5').text

'6'

In [14]:
### Script for a single page ###

#1. Creating an empty list to strore the scrapped data in

customername = []
reviewsubject = []
review = []
rating = []
likes = []
dislikes = []


#2. Extract data from individual movie names
for x in review_containers:

#If the review has name, then extract:
    if x.find('p', class_ = '_2sc7ZR _2V5EHH') is not None:
        
# The Name of the reviewer
        n = x.find('p', class_ = '_2sc7ZR _2V5EHH').text
# The subject of the review
        s = x.p.text
# The review 
        r = x.find('div', class_ = '').text
# The rating
        ra = x.find('div', class_ = '_3LWZlK _1BLPMq').text
# Likes
        l = x.find('span', class_ = '_3c3Px5').text
# Dislikes
        dl = x.find('div', class_ = '_1LmwT9 pkR4jH').find('span', class_ = '_3c3Px5').text
    
        customername.append(n)
        reviewsubject.append(s)
        review.append(r)
        rating.append(ra)
        likes.append(int(l))
        dislikes.append(int(dl))

In [15]:
customername[0:6]

['Rahul  Pandey',
 'Ajay Kumar',
 'Rahul Prasad ',
 'Avi Nash',
 'Tara singh mehra',
 'Narendar Redyy']

SCRIPT FOR MULTIPLE PAGES

Scraping multiple pages is a little challenge. We can do this in our one-page script by doing one more thing:

1) Making all the requests we want from within the loop 

Goal: To scrap all the review pages

In [16]:
headers = {"Accept-language":"en-US, en;q = 0.5"}

In [19]:
# Creating parameter that can change the page by creating a list of page length which will help in the loop to go upto the last pages 

import numpy as np

page_len =np.arange(1,213).tolist()

print(page_len)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212]


In [20]:
#1. Creating an empty list to strore the scrapped data in

customer_name = []
review_subject = []
review = []
rating = []
likes = []
dislikes = []

#2. Extract data from individual movie names
for x in page_len:

#Creating the URL:
    url1 = 'https://www.flipkart.com/apple-iphone-14-starlight-128-gb/product-reviews/itm3485a56f6e676?pid=MOBGHWFHABH3G73H&lid=LSTMOBGHWFHABH3G73HVXY5AV&marketplace=FLIPKART&page=' + str(x)
    
    # Recording the response
    response1 = get(url1, verify = False)
    
    # Creating the BeautifulSoup object
    
    page_html = BeautifulSoup(response1.text, 'html.parser')
    
    # Creating the container
    
    container_final = page_html.find_all('div', class_ = 'col _2wzgFH K0kLPL')
    
    #Extracting data from individual movie names
    for container in container_final:
        if container.find('div', class_ = '_3LWZlK _1BLPMq') is not None:
        
# The Name of the reviewer
            names = container.find('p', class_ = '_2sc7ZR _2V5EHH').text
# The subject of the review
            subject = container.p.text
# The review 
            reviews = container.find('div', class_ = '').text
# The rating
            ratings = container.find('div', class_ = '_3LWZlK _1BLPMq').text
# Likes
            like = container.find('span', class_ = '_3c3Px5').text
# Dislikes
            dislike = container.find('div', class_ = '_1LmwT9 pkR4jH').find('span', class_ = '_3c3Px5').text
    
            customer_name.append(names)
            review_subject.append(subject)
            review.append(reviews)
            rating.append(ratings)
            likes.append(int(like))
            dislikes.append(int(dislike))

C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urll

C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urll

C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urll

C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urll

C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urll

C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urll

C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urll

C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urll

C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.flipkart.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\RL465FV\Anaconda3\lib\site-packages\urll

In [21]:
##Converting into dataframe
import pandas as pd
df_reviews = pd.DataFrame({'name_of_customer' : customer_name,
                      'subject_review' : review_subject,
                      'review_by_customer': review,
                      'rating_by_customer': rating,
                      'no_of_likes': likes,
                      'no_of_dislike': dislikes})

df_reviews.tail()

,name_of_customer,subject_review,review_by_customer,rating_by_customer,no_of_likes,no_of_dislike
1406,Babu Ganesh,Perfect product!,Value for moneyREAD MORE,5,0,0
1407,Omkar Singh,Fabulous!,Phone look is really attractive and little bit...,5,0,0
1408,Aman Thakur,Fabulous!!! Worth Buying,The word APPLE says it all. Though very good c...,5,0,0
1409,Anil Salunkhe,Must buy!,I like itREAD MORE,5,0,0
1410,Nirav Nishit,Brilliant,Phone is really good to useREAD MORE,5,0,0


In [22]:
df_reviews.to_excel(r"C:\Users\RL465FV\OneDrive - EY\Desktop\My_Python\Web Scrapping\Iphone14_reviews_flipkart.xlsx", sheet_name='flipkart reviews')

TOPIC MODELLING for these reviews

Topic Modelling---- In simple words, the process of looking into large collection of documents, identifying clusters of words and grouping them together based on similarity and identifying patterns in the clusters appearing in multitude.

Latent Dirichlet Allocation (LDA) is popular algorith for Topic Modeling.

In [23]:
#!pip install sklearn
#!pip install nltk

In [24]:
# Importing libraries

import numpy as np
import pandas as pd
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation

In [25]:
# Importing the data

review = pd.read_excel(r"C:\Users\RL465FV\OneDrive - EY\Desktop\My_Python\Web Scrapping\Iphone14_reviews_flipkart.xlsx")

review.head()

,Unnamed: 0,name_of_customer,subject_review,review_by_customer,rating_by_customer,no_of_likes,no_of_dislike
0,0,Ajay Kumar,Excellent,Just amazing!READ MORE,5,1148,154
1,1,Rahul Pandey,Super!,Great picture quality. Performance is good as ...,5,91,7
2,2,Rahul Prasad,Fabulous!,Best smart phone under this price range compar...,5,9896,1503
3,3,Avi Nash,Just wow!,GoodREAD MORE,5,3639,537
4,4,Tara singh mehra,Great product,Nice camera but battery drain fast specially o...,5,2028,293


In [26]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RL465FV\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
# We will be working on review_by_customer column

# Selecting only words

df = review[['review_by_customer']]

# Cleaning the reviews

#1 Converting into lower case and removing all the characters & symbols
df['clean_text'] = df['review_by_customer'].str.lower().str.replace('[^a-z]', ' ')
df.head()

C:\Users\RL465FV\AppData\Local\Temp\ipykernel_27128\3352568151.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_text'] = df['review_by_customer'].str.lower().str.replace('[^a-z]', ' ')
C:\Users\RL465FV\AppData\Local\Temp\ipykernel_27128\3352568151.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['review_by_customer'].str.lower().str.replace('[^a-z]', ' ')


,review_by_customer,clean_text
0,Just amazing!READ MORE,just amazing read more
1,Great picture quality. Performance is good as ...,great picture quality performance is good as ...
2,Best smart phone under this price range compar...,best smart phone under this price range compar...
3,GoodREAD MORE,goodread more
4,Nice camera but battery drain fast specially o...,nice camera but battery drain fast specially o...


In [28]:
#2 Removing the stop words

stop = stopwords.words('english')

def sw(text):
    text = [word for word in text.split() if word not in stop]
    # joining the list of words with space separator
    return " ".join(text)

df['split_text'] = df['clean_text'].apply(sw)
df.head()

C:\Users\RL465FV\AppData\Local\Temp\ipykernel_27128\3286156346.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['split_text'] = df['clean_text'].apply(sw)


,review_by_customer,clean_text,split_text
0,Just amazing!READ MORE,just amazing read more,amazing read
1,Great picture quality. Performance is good as ...,great picture quality performance is good as ...,great picture quality performance good always ...
2,Best smart phone under this price range compar...,best smart phone under this price range compar...,best smart phone price range compare phones se...
3,GoodREAD MORE,goodread more,goodread
4,Nice camera but battery drain fast specially o...,nice camera but battery drain fast specially o...,nice camera battery drain fast specially video...


In [29]:
# Get rid of words which are of 3 or less than 3 letters

def lw(x):
    x = [word for word in x.split() if len(word)>3]
    return " ". join(x)

df['clean_final'] = df['split_text'].apply(lw)

df.head()

C:\Users\RL465FV\AppData\Local\Temp\ipykernel_27128\588116809.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_final'] = df['split_text'].apply(lw)


,review_by_customer,clean_text,split_text,clean_final
0,Just amazing!READ MORE,just amazing read more,amazing read,amazing read
1,Great picture quality. Performance is good as ...,great picture quality performance is good as ...,great picture quality performance good always ...,great picture quality performance good always ...
2,Best smart phone under this price range compar...,best smart phone under this price range compar...,best smart phone price range compare phones se...,best smart phone price range compare phones ov...
3,GoodREAD MORE,goodread more,goodread,goodread
4,Nice camera but battery drain fast specially o...,nice camera but battery drain fast specially o...,nice camera battery drain fast specially video...,nice camera battery drain fast specially video...


In [30]:
# Creating Tfidf vectoriser and DTM

tfid_vec = TfidfVectorizer()

# Fitting the vectoriser

tfid_vec.fit(df['clean_final'])

# Creating the DTM matrix

X = tfid_vec.fit_transform(df['clean_final'])

X

<1411x1535 sparse matrix of type '<class 'numpy.float64'>'
	with 7322 stored elements in Compressed Sparse Row format>

In [31]:
# Building LDA model with sklearn

lda_model = LatentDirichletAllocation(n_components=5, max_iter=10)

lda_output = lda_model.fit_transform(X)

print(lda_model) # Model attributes

LatentDirichletAllocation(n_components=5)


In [32]:
# How to see the document topic in each document ?

# Create the column names of documnent-topic matrix

topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# Create the index names of Document-topic matrix
docnames = ["Doc " + str(i) for i in range(len(doc))]

# Make the pandas dataframe with cells containing the probabilities of each document to classify under each topic

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns = topicnames, index = docnames)

# Get dominant topic for each document

dominant_topic = np.argmax(df_document_topic.values, axis = 1)

df_document_topic['dominant_topic'] = dominant_topic

df_dominant_topic.head()

NameError: name 'doc' is not defined